In [1]:
struct Foo
    bar
    baz
end

In [2]:
foo = Foo(1, 2)

Foo(1, 2)

In [3]:
foo.bar

1

In [4]:
foo.baz

2

In [5]:
Foo(x) = Foo(x, x)

Foo

In [6]:
Foo() = Foo(0)

Foo

In [7]:
Foo()

Foo(0, 0)

In [8]:
struct OrderedPair
    x::Real
    y::Real
    OrderedPair(x, y) = x > y ? error("out of order") : new(x, y)
end

In [9]:
OrderedPair(1, 2)

OrderedPair(1, 2)

In [10]:
OrderedPair(2, 1)

LoadError: out of order

In [11]:
struct Foo
    bar
    baz
    Foo(bar, baz) = new(bar, baz)
end

In [12]:
struct T1
    x::Int64
end

In [13]:
struct T2
    x::Int64
    T2(x) = new(x)
end

In [14]:
T1(1)

T1(1)

In [15]:
T2(1)

T2(1)

In [16]:
mutable struct SelfReferential
    obj::SelfReferential
    SelfReferential() = (x = new(); x.obj = x)
end

In [17]:
x = SelfReferential()

SelfReferential(SelfReferential(#= circular reference @-1 =#))

In [18]:
mutable struct Incomplete
    data
    Incomplete() = new()
end


In [19]:
z = Incomplete()

Incomplete(#undef)

In [20]:
z.data

LoadError: UndefRefError: access to undefined reference

In [21]:
mutable struct Lazy
    data
    Lazy(v) = complete_me(new(), v)
end

In [22]:
Lazy(5)

LoadError: UndefVarError: complete_me not defined

Parametric Constructors

In [23]:
struct Point{T<:Real}
    x::T
    y::T
end

In [24]:
Point(1,2)

Point{Int64}(1, 2)

In [25]:
Point(1.0, 2.5)

Point{Float64}(1.0, 2.5)

In [27]:
struct Point{T<:Real}
    x::T
    y::T
    Point{T}(x,y) where {T<:Real} = new(x,y)
end

In [28]:
Point(x::T, y::T) where {T<:Real} = Point{T}(x,y)

Point

In [29]:
Point(x::Int64, y::Float64) = Point(convert(Float64,x), y)

Point

In [30]:
p = Point(1, 2.5)

Point{Float64}(1.0, 2.5)

In [31]:
Point(x::Real, y::Real) = Point(promote(x, y)...)

Point

In [32]:
Point(1.5, 2)

Point{Float64}(1.5, 2.0)

In [33]:
Point(1, 1//2)

Point{Rational{Int64}}(1//1, 1//2)

In [34]:
Point(1.0,1//2)

Point{Float64}(1.0, 0.5)

In [55]:
mutable struct OurRational{T<:Integer} <: Real
    num::T
    den::T
    function OurRational{T}(num::T, den::T) where T<:Integer
        if num == 0 && den == 0
            error("invalid rational: 0//0")
        end
        g = gcd(den, num)
        num = div(num, g)
        den = div(den, g)
        print(new(num, den))
        x = new()
        x.num = num
        x.den = den
        return x
    end
end
        

In [58]:
OurRational(n::T, d::T) where {T<:Integer} = OurRational{T}(n,d) 

OurRational